In [1]:
# for similarity need to download larger english word library with the below in gitbash 
# python -m spacy download en_core_web_lg

In [2]:
import numpy as np
import pandas as pd
import spacy
import pyspark
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()

In [4]:
file = pd.read_csv("joined_data_v1.csv")

In [5]:
file = pd.DataFrame(file)
file.head()

,id,imdb_id,title,release_date,spoken_languages,genres1,genres2,genres3,genres4,genres5,...,director,poster_link,production_company,vote_average,vote_average (%),Internet_Movie_Database_rating,Rotten_Tomatoes_rating,Metacritic_rating,rating_average,writer
0,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
1,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
2,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
3,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
4,296524,tt1860357,Deepwater Horizon,9/29/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Action,NaN,NaN,NaN,...,Peter Berg,https://m.media-amazon.com/images/M/MV5BOTEzND...,Lionsgate,6.5,65%,71%,83%,68%,72%,"Matthew Michael Carnahan (screenplay by), Matt..."


In [6]:
file.drop_duplicates(keep='first', inplace=True)

In [7]:
file.head()

,id,imdb_id,title,release_date,spoken_languages,genres1,genres2,genres3,genres4,genres5,...,director,poster_link,production_company,vote_average,vote_average (%),Internet_Movie_Database_rating,Rotten_Tomatoes_rating,Metacritic_rating,rating_average,writer
0,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
2,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
4,296524,tt1860357,Deepwater Horizon,9/29/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Action,NaN,NaN,NaN,...,Peter Berg,https://m.media-amazon.com/images/M/MV5BOTEzND...,Lionsgate,6.5,65%,71%,83%,68%,72%,"Matthew Michael Carnahan (screenplay by), Matt..."
6,283366,tt1935859,Miss Peregrine's Home for Peculiar Children,9/28/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Fantasy,Adventure,NaN,NaN,...,Tim Burton,https://m.media-amazon.com/images/M/MV5BMTU0Nz...,20th Century Fox,6.5,65%,67%,65%,57%,64%,Ransom Riggs (based upon the novel written by)...
8,284564,tt3835080,31,9/15/2016,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Horror,Thriller,NaN,NaN,NaN,...,Rob Zombie,https://m.media-amazon.com/images/M/MV5BNTE0NT...,Bow and Arrow Entertainment,4.9,49%,51%,49%,35%,46%,Rob Zombie


In [8]:
overview_df2 = file[["title", "overview"]].copy()
overview_df2.head()

,title,overview
0,The Birth of a Nation,"Nat Turner, a former slave in America, leads a..."
2,Sully,"On 15 January 2009, the world witnessed the 'M..."
4,Deepwater Horizon,A story set on the offshore drilling rig Deepw...
6,Miss Peregrine's Home for Peculiar Children,A teenager finds himself transported to an isl...
8,31,ROB ZOMBIE'S S P O O K H A U S 31


In [9]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [10]:
sentenceData = spark.createDataFrame(overview_df2)
sentenceData.show() 

+--------------------+--------------------+
|               title|            overview|
+--------------------+--------------------+
|The Birth of a Na...|Nat Turner, a for...|
|               Sully|On 15 January 200...|
|   Deepwater Horizon|A story set on th...|
|Miss Peregrine's ...|A teenager finds ...|
|                  31|ROB ZOMBIE'S  S P...|
|             Snowden|CIA employee Edwa...|
|         Blair Witch|Students on a cam...|
|The Magnificent S...|Seven gun men in ...|
|              Morgan|A corporate risk-...|
|      The Dark Tower|The last Gunsling...|
|          Nine Lives|A stuffy business...|
|       Don't Breathe|A group of teens ...|
|       Suicide Squad|From DC Comics co...|
|Kubo and the Two ...|Kubo mesmerizes t...|
|             Ben-Hur|A falsely accused...|
|            War Dogs|Based on the true...|
|  Hell or High Water|A divorced dad an...|
|          Anthropoid|In December 1941,...|
|       Pete's Dragon|Pete is a mysteri...|
|            Rock Dog|When a rad

In [11]:
tokenizer = Tokenizer(inputCol="overview", outputCol="words")
tokenizer

Tokenizer_3948f8fd01f3

In [12]:
# Transform and show DataFrame
tokenized = tokenizer.transform(sentenceData)
tokenized.show(truncate=False)

+-------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title                                      |overview               

In [13]:
# Instantiate Remover
remover = StopWordsRemover(inputCol="words", outputCol="stopwords")

In [14]:
# Transform and show data
tokenized = remover.transform(tokenized)
tokenized.show(truncate=False)

+-------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------

In [15]:
tokenized.show(truncate=True)

+--------------------+--------------------+--------------------+--------------------+
|               title|            overview|               words|           stopwords|
+--------------------+--------------------+--------------------+--------------------+
|The Birth of a Na...|Nat Turner, a for...|[nat, turner,, a,...|[nat, turner,, fo...|
|               Sully|On 15 January 200...|[on, 15, january,...|[15, january, 200...|
|   Deepwater Horizon|A story set on th...|[a, story, set, o...|[story, set, offs...|
|Miss Peregrine's ...|A teenager finds ...|[a, teenager, fin...|[teenager, finds,...|
|                  31|ROB ZOMBIE'S  S P...|[rob, zombie's, ,...|[rob, zombie's, ,...|
|             Snowden|CIA employee Edwa...|[cia, employee, e...|[cia, employee, e...|
|         Blair Witch|Students on a cam...|[students, on, a,...|[students, campin...|
|The Magnificent S...|Seven gun men in ...|[seven, gun, men,...|[seven, gun, men,...|
|              Morgan|A corporate risk-...|[a, corpora

# Starting Spacy Similarity code

In [16]:
# Load English tokenizer, tagger, parser, NER and word vectors
#MAKE SURE THIS IS LARGE LIBRARY

nlp = spacy.load("en_core_web_lg")
import string
punctuations = string.punctuation


In [17]:
overview1 =str(tokenized.select("stopwords").collect()[0])
clean_overview1 = overview1[16:]

overview2 = str(tokenized.select("stopwords").collect()[1])
clean_overview2 = overview2[16:]


In [18]:
clean_overview1

"nat', 'turner,', 'former', 'slave', 'america,', 'leads', 'liberation', 'movement', '1831', 'free', 'african-americans', 'virgina', 'results', 'violent', 'retaliation', 'whites.'])"

In [19]:
clean_overview2

'15\', \'january\', \'2009,\', \'world\', \'witnessed\', "\'miracle", "hudson\'", \'captain\', "\'sully\'", \'sullenberger\', \'glided\', \'disabled\', \'plane\', \'onto\', \'frigid\', \'waters\', \'hudson\', \'river,\', \'saving\', \'lives\', \'155\', \'aboard.\', \'however,\', \'even\', \'sully\', \'heralded\', \'public\', \'media\', \'unprecedented\', \'feat\', \'aviation\', \'skill,\', \'investigation\', \'unfolding\', \'threatened\', \'destroy\', \'reputation\', \'career.\'])'

In [20]:
docA = nlp(clean_overview1)
docB = nlp(clean_overview2)

print(docA.similarity(docB))
type(docA) , type(clean_overview1)

0.984993000906136


(spacy.tokens.doc.Doc, str)

In [21]:
print(docB)

15', 'january', '2009,', 'world', 'witnessed', "'miracle", "hudson'", 'captain', "'sully'", 'sullenberger', 'glided', 'disabled', 'plane', 'onto', 'frigid', 'waters', 'hudson', 'river,', 'saving', 'lives', '155', 'aboard.', 'however,', 'even', 'sully', 'heralded', 'public', 'media', 'unprecedented', 'feat', 'aviation', 'skill,', 'investigation', 'unfolding', 'threatened', 'destroy', 'reputation', 'career.'])


In [22]:
overviewC = str(tokenized.select("stopwords").collect()[2])
clean_overviewC = overviewC[16:]
docC = nlp(clean_overviewC)

In [23]:
clean_overviewC

"story', 'set', 'offshore', 'drilling', 'rig', 'deepwater', 'horizon,', 'exploded', 'april', '2010', 'created', 'worst', 'oil', 'spill', 'u.s.', 'history.'])"

In [24]:
print(docA.similarity(docC))

0.9805625913523139


In [25]:
print(docB.similarity(docC))

0.9873667102432259


## adding columns for similarities

In [26]:
from pyspark.sql.functions import lit

In [38]:
ll = []
for f in tokenized.collect():
   
    overview_row = str(f.stopwords)
    #print(overview_row)
    doc_row = nlp(overview_row)

    compare = docA.similarity(doc_row)
    ll.append({'Title': f.title, 'Similarity': compare })
# sample = tokenized.withColumn('similarity', lit(str(ll)))
# ll   


In [39]:
ll

[{'Title': 'The Birth of a Nation', 'Similarity': 0.9995973840399037},
 {'Title': 'Sully', 'Similarity': 0.9849470144479944},
 {'Title': 'Deepwater Horizon', 'Similarity': 0.9805782738804641},
 {'Title': "Miss Peregrine's Home for Peculiar Children",
  'Similarity': 0.9815931520061407},
 {'Title': '31', 'Similarity': 0.9608165636588171},
 {'Title': 'Snowden', 'Similarity': 0.9767509005577233},
 {'Title': 'Blair Witch', 'Similarity': 0.9741712529231993},
 {'Title': 'The Magnificent Seven', 'Similarity': 0.9852484058711272},
 {'Title': 'Morgan', 'Similarity': 0.9723964056257384},
 {'Title': 'The Dark Tower', 'Similarity': 0.9870985405152113},
 {'Title': 'Nine Lives', 'Similarity': 0.9673087506469586},
 {'Title': "Don't Breathe", 'Similarity': 0.9805259498839797},
 {'Title': 'Suicide Squad', 'Similarity': 0.9876107808980823},
 {'Title': 'Kubo and the Two Strings', 'Similarity': 0.9812733500570661},
 {'Title': 'Ben-Hur', 'Similarity': 0.9814676541153275},
 {'Title': 'War Dogs', 'Similarity

In [40]:
df_simi = pd.DataFrame(ll)

In [41]:
# from pyspark.sql import SQLContext
df_simi.head()

,Similarity,Title
0,0.999597,The Birth of a Nation
1,0.984947,Sully
2,0.980578,Deepwater Horizon
3,0.981593,Miss Peregrine's Home for Peculiar Children
4,0.960817,31


In [43]:
df_simi.sort_values(by=['Similarity'], inplace=True, ascending=False)
df_simi.head(10)

,Similarity,Title
0,0.999597,The Birth of a Nation
39,0.991051,Free State of Jones
144,0.990280,Loving
142,0.988981,Hidden Figures
170,0.988760,The Lost City of Z
90,0.988724,Whiskey Tango Foxtrot
79,0.988641,A Hologram for the King
156,0.988597,Allied
115,0.988571,Pride and Prejudice and Zombies
120,0.988391,Eddie the Eagle


In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

In [ ]:
from pyspark.sql.types import *
cSchema = StructType([StructField("Similarity", ArrayType(FloatType()))])
b= spark.createDataFrame(ll, FloatType()).toDF("x")
#b = spark.createDataFrame(ll, schema = cSchema)

tokenized = tokenized.withColumn("row_idx", monotonically_increasing_id())
b = b.withColumn("row_idx", monotonically_increasing_id())

final_df = tokenized.join(b, tokenized.row_idx == b.row_idx).\
            drop("row_idx")
final_df.show()

In [ ]:
sample.show()

## Removed punctuation

In [ ]:
docI = nlp(clean_overview1)
tokenI = [tok.lemma_.lower() for tok in docI]
tokenI = [tok for tok in tokenI if tok not in punctuations]
# tokenI = ' '.join(tokenI)

print(tokenI)

In [ ]:
docII = nlp(clean_overview2)
tokenII = [tok.lemma_.lower().strip() for tok in docII]
tokenII = [tok for tok in tokenII if tok not in punctuations]

print(tokenII)

In [ ]:
for f in tokenized.collect():
    doc_r=''
    overview_r = str(f.stopwords)
    print(overview_r)
    doc_r = nlp(overview_r)
    comp = docI.similarity(doc_r)
    samp = tokenized.withColumn('similarity', lit(str(compare)))

In [ ]:
samp.show()

In [ ]:
sim = docI.similarity(docII)
sim

In [ ]:
sim1 = overview1.similarity(overview2)
sim1

In [ ]:
text_compareX = []
docx = nlp(clean_overview1, disable=['parser'])

tokensx = [tok.lemma_ for tok in docx if tok not in punctuations]
tokensx = ' '.join(tokensx)
text_compareX.append(tokensx)

text_compareX

In [ ]:
text_compare1 = []
doc1 = nlp(clean_overview1, disable=['parser'])

tokens1 = [tok.lemma_.lower().strip() for tok in doc1]
tokens1 = [tok for tok in tokens1 if tok not in punctuations]
tokens1 = ' '.join(tokens1)
text_compare1.append(tokens1)

text_compare1

In [ ]:
simil = tokens1.similarity(tokens2)
simil

In [ ]:
text_compare2 = []
doc2 = nlp(clean_overview2, disable=['parser'])

tokens2 = [tok.lemma_.lower().strip() for tok in doc2]
tokens2 = [tok for tok in tokens2 if tok not in punctuations]
tokens2 = ' '.join(tokens2)
text_compare2.append(tokens2)

text_compare2

In [ ]:
comp = text_compare1.similarity(text_compare2)
comp

In [ ]:
length = len(text_compare)
i = 0

while i < length: 
    for t[0+i] in text_compare:
        for t[1+i] in text_compare:
            print(t[0+i].text, t[1+i].text, t[0+i].similarity(t[1+i]))
    

In [ ]:
for token in tokens1:
    for token in tokens2:
        print(tokens1, tokens2, tokens1.similarity(tokens2))